In [15]:
import numpy as np
import pandas as pd
from gurobipy import *

In [16]:
file_name = 'dataset.xlsx'

density = pd.read_excel(file_name, 'Density')
locations = len(density['G'])

popDensity = np.asarray(density['density'])

rate = pd.read_excel(file_name, 'Vaccination_Rate')
vacRate = np.asarray(rate.iloc[:,2:])

scrNeed = pd.read_excel(file_name, 'Screen_needed')
screenNeeded = np.asarray(scrNeed.iloc[:,2:])

setsta = pd.read_excel(file_name, 'Actual_screened_station')
setStation = np.asarray(setsta.iloc[:,2])

limit = 300

date = int(screenNeeded.shape[1])
Q = sum(setStation)

In [17]:
final = Model("final")

In [18]:
x=[]
for i in range(locations):
    x.append([])
    for t in range(date):
        x[i].append(final.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x" +  str(i+1) +'_'+ str(t+1)))

In [19]:
y=[]
for i in range(locations):
    y.append([])
    for k in range(date):
        y[i].append(final.addVar(vtype = GRB.CONTINUOUS, name = "y" +  str(i+1) +'_'+ str(k+1)))

In [20]:
final.setObjective(
    
quicksum( popDensity[i]*quicksum((5/4)*((1-vacRate[i][t])**2)*(y[i][t]) for t in range(date)) for i in range(locations))
    
    , GRB.MINIMIZE) 

In [21]:
final.addConstr( quicksum( quicksum(x[i][t] for i in range(locations)) for t in range(date) ) <= Q , "cons1")

final.addConstrs((y[i][t] >=  y[i][t-1] + screenNeeded[i][t] - (limit*quicksum(x[i][k] for k in range(t)))
                  for i in range(locations) for t in range(date) if t > 0), "cons2")
final.addConstrs((y[i][t] >=  0 + screenNeeded[i][t] - (limit*quicksum(x[i][k] for k in range(t)))
                  for i in range(locations) for t in range(date) if t == 0), "cons3")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (7, 0): <gurobi.Constr *Awaiting Model Update*>,
 (8, 0): <gurobi.Constr *Awaiting Model Update*>,
 (9, 0): <gurobi.Constr *Awaiting Model Update*>,
 (10, 0): <gurobi.Constr *Awaiting Model Update*>,
 (11, 0): <gurobi.Constr *Awaiting Model Update*>,
 (12, 0): <gurobi.Constr *Awaiting Model Update*>,
 (13, 0): <gurobi.Constr *Awaiting Model Update*>,
 (14, 0): <gurobi.Constr *Awaiting Model Update*>,
 (15, 0): <gurobi.Constr *Awaiting Model Update*>,
 (16, 0): <gurobi.Constr *Awaiting Model Update*>,
 (17, 0): <gurobi.Constr *Awaiting Model Update*>,
 (18, 0): <gurobi.Constr *Awaiting Model Update*>,
 (19, 0): <gurobi.Constr *Awaiting Model 

In [22]:
final.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 616 rows, 1230 columns and 6109 nonzeros
Model fingerprint: 0xb7b83068
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [2e+01, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+03]
Presolve removed 110 rows and 220 columns
Presolve time: 0.06s
Presolved: 506 rows, 1010 columns, 5202 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4361208e+08   1.854000e+03   0.000000e+00      0s
     387    2.0995757e+08   0.000000e+00   0.000000e+00      0s

Solved in 387 iterations and 0.13 seconds
Optimal objective  2.099575743e+08


In [23]:
print("z* = ", final.objVal) 

z* =  209957574.25214213


In [24]:
x

[[<gurobi.Var x1_1 (value 0.41333333333333333)>,
  <gurobi.Var x1_2 (value 0.0)>,
  <gurobi.Var x1_3 (value 0.0)>,
  <gurobi.Var x1_4 (value 0.0)>,
  <gurobi.Var x1_5 (value 0.0)>,
  <gurobi.Var x1_6 (value 0.014166666666666666)>,
  <gurobi.Var x1_7 (value 0.0)>,
  <gurobi.Var x1_8 (value 0.0)>,
  <gurobi.Var x1_9 (value 0.0)>,
  <gurobi.Var x1_10 (value 0.0)>,
  <gurobi.Var x1_11 (value 0.0)>,
  <gurobi.Var x1_12 (value 0.0)>,
  <gurobi.Var x1_13 (value 0.0)>,
  <gurobi.Var x1_14 (value 0.0)>,
  <gurobi.Var x1_15 (value 0.0)>],
 [<gurobi.Var x2_1 (value 0.20666666666666667)>,
  <gurobi.Var x2_2 (value 0.0)>,
  <gurobi.Var x2_3 (value 0.2733333333333333)>,
  <gurobi.Var x2_4 (value 0.0)>,
  <gurobi.Var x2_5 (value 0.0)>,
  <gurobi.Var x2_6 (value 0.13666666666666666)>,
  <gurobi.Var x2_7 (value 0.0)>,
  <gurobi.Var x2_8 (value 0.17)>,
  <gurobi.Var x2_9 (value 0.0)>,
  <gurobi.Var x2_10 (value 0.0)>,
  <gurobi.Var x2_11 (value 0.0)>,
  <gurobi.Var x2_12 (value 0.0)>,
  <gurobi.Var x2_1